In [8]:
import pandas as pd
import numpy as nmp
import requests
import os
import json
from io import StringIO
import time

In [2]:
noaa_api = os.getenv('noaa_api')

In [3]:
coastal_api = os.getenv('coastal_api')

In [4]:
r = requests.get('http://httpbin.org/user-agent')
useragent = json.loads(r.text)['user-agent']

In [5]:
ocmd = pd.read_csv('OCMD_beachrenourishment.csv')
ocmd['station_id'] = '8570283'

## National Bouy Data Center

station ids: https://www.ndbc.noaa.gov/to_station.shtml

In [6]:
dbdc_url = 'https://www.ndbc.noaa.gov/data/realtime2/{id}.txt'

In [7]:
station_ids = [
    'AAMC1', 'ACYN4', 'ADKA2', 'AGCM4', 'ALIA2', 'ALXN6', 'AMRL1', 'ANTA2', 'ANVC1', 'APAM2',
    'APCF1', 'APRP7', 'ASTO3', 'ATGM1', 'ATKA2', 'BARA9', 'BATN6', 'BEPB6', 'BFTN7', 'BHBM3',
    'BISM2', 'BKTL1', 'BLTM2', 'BRHC3', 'BUFN6', 'BYGL1', 'BZBM3', 'CAMM2', 'CAPL1', 'CARL1',
    'CASM1', 'CBBV2', 'CCUF1', 'CECC1', 'CFWM1', 'CHAO3', 'CHAV3', 'CHBV2', 'CHSV3', 'CHTM3',
    'CHTS1', 'CHYW1', 'CKYF1', 'CLBP4', 'CMAN4', 'CMTI2', 'CNDO1', 'CRVA2', 'CTGM1', 'CWBF1',
    'DILA1', 'DKCM6', 'DTLM4', 'DUKN7', 'DULM5', 'EBSW1', 'ELFA2', 'EREP1', 'ERTF1', 'ESPP4',
    'FAIO1', 'FCGT2', 'FMRF1', 'FOXR1', 'FPKG1', 'FPST2', 'FPTT2', 'FRCB6', 'FRDF1', 'FRDW1',
    'FREL1', 'FRPS1', 'FRWL1', 'FTGM4', 'FTPC1', 'GBWW3', 'GCVF1', 'GDMM5', 'GISL1', 'GNJT2',
    'GPOM6', 'GPST2', 'GTOT2', 'GWPM6', 'HBYC1', 'HCGN7', 'HLNM4', 'HMDO3', 'HRBM4', 'HRVC1',
    'ICAC1', 'IIWC1', 'ILOH1', 'ITKA2', 'JMPN7', 'JNEA2', 'JXUF1', 'KDAA2', 'KECA2', 'KGCA2',
    'KLIH1', 'KPTN6', 'KPTV2', 'KWHH1', 'KWJP8', 'KWNW3', 'KYWF1', 'LABL1', 'LAMV3', 'LAPW1',
    'LCLL1', 'LDTM4', 'LJAC1', 'LKWF1', 'LOPW1', 'LPNM4', 'LTBV3', 'LTRM4', 'LWSD1', 'LWTV2',
    'MACM4', 'MBET2', 'MBRM4', 'MCGM4', 'MCYF1', 'MEYC1', 'MGIP4', 'MGZP4', 'MISP4', 'MNMM4',
    'MOKH1', 'MQTT2', 'MRHO1', 'MROS1', 'MTKN6', 'MTYC1', 'MYPF1', 'NCDV2', 'NEAW1', 'NIAN6',
    'NJLC1', 'NKTA2', 'NLHC3', 'NLNC3', 'NMTA2', 'NPSF1', 'NSTP6', 'NTBC1', 'NTKM3', 'NUET2',
    'NWCL1', 'NWPR1', 'NWWH1', 'OBGN6', 'OBLA1', 'OCIM2', 'OHBC1', 'OLSA2', 'OOUH1', 'ORIN7',
    'OSGN6', 'OVIA2', 'PACF1', 'PBWM4', 'PCBF1', 'PCLF1', 'PCOC1', 'PGBP7', 'PHBP1', 'PLXA2',
    'PMOA2', 'PNLM4', 'PNLM6', 'PORO3', 'PRDA2', 'PRYC1', 'PSBM1', 'PSLC1', 'PSTL1', 'PSTN6',
    'PTAW1', 'PTIM4', 'PTIT2', 'PTWW1', 'RARM6', 'RCKM4', 'RCPT2', 'RCRN6', 'RCYF1', 'RDDA2',
    'RDYD1', 'SAPF1', 'SBEO3', 'SBLF1', 'SBPT2', 'SCIS1', 'SDBC1', 'SDHN4', 'SHBL1', 'SJNP4',
    'SKTA2', 'SLIM2', 'SMOC1', 'SNDA2', 'SNDP5', 'SWLA2', 'SWPV2', 'TESL1', 'THRO1', 'TLBO3',
    'TOKW1', 'TPBN4', 'TRDF1', 'TXPT2', 'ULAM6', 'ULRA2', 'UNLA2', 'VAKF1', 'VCAF1', 'VCVA2',
    'VDZA2', 'WAHV2', 'WAKP8', 'WASD2', 'WAVM6', 'WLON7', 'WNDV2', 'WNEM4', 'WPTW1', 'WYCM6',
    'YATA2', 'YKTV2'
]


In [79]:
dbdc_df = pd.DataFrame()
for station_id in station_ids:
    formatted_url = dbdc_url.format(id=station_id)
    r = requests.get(formatted_url)
    if r.status_code == 200:
        df = pd.read_csv(formatted_url, delim_whitespace = True, header = None)
        df.drop(df.index[0], inplace = True)
        df['Station'] = station_id
        dbdc_df = pd.concat([dbdc_df, df], ignore_index = True)
    else: 
        print(f"Failed to retrieve data for {station_id}: {r.status_code}")
    

Failed to retrieve data for BARA9: 404
Failed to retrieve data for BEPB6: 404
Failed to retrieve data for CBBV2: 404
Failed to retrieve data for CCUF1: 404
Failed to retrieve data for CTGM1: 404
Failed to retrieve data for DKCM6: 404
Failed to retrieve data for EBSW1: 404
Failed to retrieve data for ERTF1: 404
Failed to retrieve data for FCGT2: 404
Failed to retrieve data for FPTT2: 404
Failed to retrieve data for FREL1: 404
Failed to retrieve data for FRPS1: 404
Failed to retrieve data for GCVF1: 404
Failed to retrieve data for GPOM6: 404
Failed to retrieve data for GPST2: 404
Failed to retrieve data for GWPM6: 404
Failed to retrieve data for HMDO3: 404
Failed to retrieve data for HRVC1: 404
Failed to retrieve data for IIWC1: 404
Failed to retrieve data for LABL1: 404
Failed to retrieve data for MCYF1: 404
Failed to retrieve data for MQTT2: 404
Failed to retrieve data for MTYC1: 404
Failed to retrieve data for NLNC3: 404
Failed to retrieve data for NPSF1: 404
Failed to retrieve data f

In [81]:
dbdc_df
#dbdc_df.iloc[10910:10950]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,Station
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,nmi,hPa,ft,AAMC1
1,2023,11,17,18,18,190,1.5,1.5,MM,MM,MM,MM,1013.7,15.9,MM,MM,MM,MM,MM,AAMC1
2,2023,11,17,18,12,210,1.0,1.5,MM,MM,MM,MM,1013.8,15.9,MM,MM,MM,MM,MM,AAMC1
3,2023,11,17,18,06,190,1.0,1.5,MM,MM,MM,MM,1013.9,15.9,MM,MM,MM,MM,MM,AAMC1
4,2023,11,17,18,00,200,1.0,1.0,MM,MM,MM,MM,1013.9,16.0,MM,MM,MM,+0.0,MM,AAMC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102334,2023,10,03,00,24,120,2.6,3.1,MM,MM,MM,MM,1020.3,21.8,22.1,MM,MM,MM,MM,YKTV2
2102335,2023,10,03,00,18,110,2.1,3.6,MM,MM,MM,MM,1020.2,21.9,22.1,MM,MM,MM,MM,YKTV2
2102336,2023,10,03,00,12,120,2.6,3.6,MM,MM,MM,MM,1020.2,21.9,22.1,MM,MM,MM,MM,YKTV2
2102337,2023,10,03,00,06,110,2.6,4.1,MM,MM,MM,MM,1020.2,22.0,22.1,MM,MM,MM,MM,YKTV2


## NOAA API 

In [24]:
root = 'https://www.ncei.noaa.gov/cdo-web/api/v2'
endpoint = f'/locations'

In [15]:
parameters = {'format': 'json',
            'offset': 1,
            'count': 149410,
            'limit': 25}

In [17]:
headers = {'User-Agent': useragent,
           'From': 'zrc3hc@virginia.edu',
          'token': noaa_api}

In [27]:
r = requests.get(root + endpoint,
                params = parameters,
                headers = headers)

In [31]:
r

<Response [200]>

In [15]:
id = 'GHCND:USW00093786'

In [18]:
#json.loads(r.text)

In [6]:
root = 'https://www.ncei.noaa.gov/cdo-web/api/v2/stations'

In [32]:

def get_all_stations_data():
    all_stations = []
    offset = 1
    limit = 25  # Adjust based on what the API allows
    total_stations = 149410  # Update this dynamically if possible

    while offset <= total_stations:
        params = {'limit': limit, 'offset': offset}
        response = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/stations', headers=headers, params=params)

        if response.status_code == 200:
            page_data = response.json()['results']
            all_stations.extend(page_data)
            offset += limit
        else:
            print(f"Error at offset {offset}: {response.status_code} - {response.text}")
            break  # Exit loop on error

        # Sleep to avoid hitting rate limits
        time.sleep(1)

    return all_stations

headers = {'User-Agent': useragent, 'From': 'zrc3hc@virginia.edu', 'token': noaa_api}
station_data = get_all_stations_data()
df_stations = pd.DataFrame(station_data)

print(f"Total rows in DataFrame: {df_stations.shape[0]}")



Error at offset 6601: 503 - <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>

Total rows in DataFrame: 6600


In [63]:

def get_all_stations_data(start_offset):
    all_stations = []
    limit = 25  # Adjust based on what the API allows
    total_stations = 149410  # Update this dynamically if possible

    for offset in range(start_offset, total_stations + 1, limit):
        params = {'limit': limit, 'offset': offset}
        response = requests.get('https://www.ncei.noaa.gov/cdo-web/api/v2/stations', headers=headers, params=params)

        if response.status_code == 200:
            page_data = response.json()['results']
            all_stations.extend(page_data)
        else:
            print(f"Error at offset {offset}: {response.status_code} - {response.text}")
            break  # Exit loop on error

        # Sleep to avoid hitting rate limits
        time.sleep(1)

    return all_stations

headers = {'User-Agent': useragent, 'From': 'zrc3hc@virginia.edu', 'token': noaa_api}

# Start from the offset 6601
station_data2 = get_all_stations_data(8250)

Error at offset 9200: 503 - <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>503 Service Unavailable</title>
</head><body>
<h1>Service Unavailable</h1>
<p>The server is temporarily unable to service your
request due to maintenance downtime or capacity
problems. Please try again later.</p>
<p>Additionally, a 503 Service Unavailable
error was encountered while trying to use an ErrorDocument to handle the request.</p>
</body></html>



In [62]:
df_stations1 = pd.DataFrame(station_data1)
df_stations1

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,7.6,1947-10-01,2014-12-01,30.47140,"ABITA SPRINGS 1 SW, LA US",0.2786,COOP:160016,METERS,-90.04360
1,9.1,1973-04-01,2015-11-01,30.43720,"ABITA SPRINGS FIRE TOWER, LA US",0.9883,COOP:160021,METERS,-90.05190
2,14.9,1951-10-01,1959-02-01,31.28333,"ACME, LA US",1.0000,COOP:160028,METERS,-91.81667
3,67.1,1982-10-01,1985-05-01,31.80000,"AIMWELL FIRE TOWER, LA US",1.0000,COOP:160045,METERS,-91.96667
4,26.5,1858-07-01,2015-11-01,31.32050,"ALEXANDRIA, LA US",0.6697,COOP:160098,METERS,-92.46110
...,...,...,...,...,...,...,...,...,...
1645,612.6,1957-09-01,2005-07-01,47.61810,"ISABELLA 1 W, MN US",0.9252,COOP:214068,METERS,-91.37530
1646,530.4,2009-11-01,2015-11-01,47.59333,"ISABELLA 14 W, MN US",1.0000,COOP:214077,METERS,-91.65139
1647,429.8,2009-10-01,2015-11-01,47.02389,"ISLAND LAKE 4 E, MN US",1.0000,COOP:214093,METERS,-92.07000
1648,418.2,1948-08-01,1995-09-01,46.98333,"ISLAND LAKE RESERVOIR, MN US",0.9806,COOP:214096,METERS,-92.23333


In [64]:
df_stations2 = pd.DataFrame(station_data2)
df_stations2

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,384.0,1936-04-01,1945-11-01,46.13333,"ISLE, MN US",0.9051,COOP:214102,METERS,-93.46667
1,391.7,1959-10-01,2015-11-01,46.31970,"ISLE 12 N, MN US",0.9125,COOP:214103,METERS,-93.50130
2,454.2,1911-05-01,2015-11-01,47.22550,"ITASCA UNIVERSITY OF MINN, MN US",0.9896,COOP:214106,METERS,-95.19190
3,274.3,1948-08-01,2015-11-01,44.65010,"JORDAN 1 SSW, MN US",0.8936,COOP:214176,METERS,-93.63560
4,365.8,1997-11-01,2015-11-01,48.40880,"KABETOGAMA, MN US",0.8573,COOP:214191,METERS,-93.04840
...,...,...,...,...,...,...,...,...,...
945,234.7,1949-05-01,1995-01-01,39.40000,"STET 1 S, MO US",0.0837,COOP:238063,METERS,-93.76667
946,281.0,1970-04-01,2015-11-01,37.69680,"STOCKTON DAM, MO US",0.9853,COOP:238082,METERS,-93.77250
947,320.0,1931-01-01,2004-11-01,38.44083,"STOVER, MO US",0.8918,COOP:238112,METERS,-92.99500
948,NaN,1948-01-01,1950-04-01,36.53333,"SUGAR CAMP LOOKOUT, MO US",0.4650,COOP:238160,NaN,-93.83333


In [19]:
r = requests.get(root, params=parameters, headers=headers)


<Response [200]>

In [26]:
data = r.json()

features = data['results']
df = pd.DataFrame(features)


In [28]:
df

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,COOP:010008,METERS,-85.24820
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,COOP:010063,METERS,-87.17838
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,COOP:010071,METERS,-87.31667
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,COOP:010116,METERS,-86.78178
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,COOP:010117,METERS,-86.88190
5,34.1,1935-05-01,1936-11-01,31.13333,"ALAGA, AL US",0.2624,COOP:010125,METERS,-85.06667
6,53.3,1940-11-01,2014-12-01,32.23220,"ALBERTA, AL US",0.9888,COOP:010140,METERS,-87.41040
7,348.1,1931-01-01,1977-06-01,34.23333,"ALBERTVILLE, AL US",0.9535,COOP:010148,METERS,-86.16667
8,201.2,1969-10-01,2015-11-01,32.93500,"ALEXANDER CITY, AL US",0.9946,COOP:010160,METERS,-85.95556
9,200.9,1942-11-01,1969-10-01,32.98333,"ALEXANDER CITY 6 NE, AL US",0.9629,COOP:010163,METERS,-85.86667


In [ ]:
records = [feature['attributes'] for feature in features]
df = pd.DataFrame(records)
df

In [34]:
root = 'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter'
parameters = {'date': 'latest',
              'station': '160012',
              'product': 'air_temperature',
              'datum': 'STND',
              'time_zone': 'gmt',
              'units': 'english',
              'format':'json'
             }

In [35]:
headers = {'User-Agent': useragent,
           'From': 'zrc3hc@virginia.edu',
          'token': noaa_api}

In [36]:
r = requests.get(root, params=parameters, headers=headers)

In [37]:
r

<Response [400]>

In [36]:
json.loads(r.text)

{'metadata': {'id': '8570283',
  'name': 'Ocean City Inlet',
  'lat': '38.3283',
  'lon': '-75.0917'},
 'data': [{'t': '2023-11-17 15:12', 'v': '60.3', 'f': '0,0,0'}]}

In [44]:
root = 'http://www.ncei.noaa.gov/cdo-web/api/v2/stations/COOP:010008'
r = requests.get(root, params=parameters, headers=headers)
json.loads(r.text)


{'elevation': 139,
 'mindate': '1948-01-01',
 'maxdate': '2014-01-01',
 'latitude': 31.5702,
 'name': 'ABBEVILLE, AL US',
 'datacoverage': 0.8813,
 'id': 'COOP:010008',
 'elevationUnit': 'METERS',
 'longitude': -85.2482}

In [59]:
root = 'http://www.ncei.noaa.gov/cdo-web/api/v2/datasets'
parameters = {
    'datasetid': 'GSOY', 
    'format': 'json',
    'limit': 50 
             }
r = requests.get(root, params=parameters, headers=headers)
json.loads(r.text)

{'metadata': {'resultset': {'offset': 1, 'count': 11, 'limit': 50}},
 'results': [{'uid': 'gov.noaa.ncdc:C00861',
   'mindate': '1763-01-01',
   'maxdate': '2023-11-27',
   'name': 'Daily Summaries',
   'datacoverage': 1,
   'id': 'GHCND'},
  {'uid': 'gov.noaa.ncdc:C00946',
   'mindate': '1763-01-01',
   'maxdate': '2023-11-01',
   'name': 'Global Summary of the Month',
   'datacoverage': 1,
   'id': 'GSOM'},
  {'uid': 'gov.noaa.ncdc:C00947',
   'mindate': '1763-01-01',
   'maxdate': '2023-01-01',
   'name': 'Global Summary of the Year',
   'datacoverage': 1,
   'id': 'GSOY'},
  {'uid': 'gov.noaa.ncdc:C00345',
   'mindate': '1991-06-05',
   'maxdate': '2023-12-01',
   'name': 'Weather Radar (Level II)',
   'datacoverage': 0.95,
   'id': 'NEXRAD2'},
  {'uid': 'gov.noaa.ncdc:C00708',
   'mindate': '1994-05-20',
   'maxdate': '2023-11-29',
   'name': 'Weather Radar (Level III)',
   'datacoverage': 0.95,
   'id': 'NEXRAD3'},
  {'uid': 'gov.noaa.ncdc:C00821',
   'mindate': '2010-01-01',
   